<a href="https://colab.research.google.com/github/anson1788/AiCardBackend/blob/main/opcg_turn_though.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#!pip install llama_index
#!pip install llama-index-llms-groq
#!pip install llama-index-llms-openai
#!pip install llama-index-agent-openai
from llama_index.llms.groq import Groq
import random
from pydantic import Field

cardListContext ="""

#Card effect and details list
id:ST01-001
Name:Monkey.D.Luffy
Card type:Leader
Life:5
Attribute:Strike
Power:5000
Counter:-
Color:Red
Type:Supernovas/Straw Hat Crew
Card Effect:
[Activate: Main] [Once Per Turn] Give this Leader or 1 of your Characters up to 1 rested DON!! card.

id:ST01-002
Name:Usopp
Card type:Character
Cost:2
Attribute:Ranged
Power:2000
Counter:1000
Color:Red
Type:Straw Hat Crew
Card Effect:
[DON!! x2] [When Attacking] Your opponent cannot activate a [Blocker] Character that has 5000 or more power during this battle.
Trigger:
Play this card.

id:ST01-003
Name:Karoo
Card type:Character
Cost:1
Attribute:Strike
Power:3000
Counter:1000
Color:Red
Type:Animal/Alabasta
Card Effect:

id:ST01-004
Name:Sanji
Card type:Character
Cost:2
Attribute:Strike
Power:4000
Counter:-
Color:Red
Type:Straw Hat Crew
Card Effect:
[DON!! x2] This Character gains [Rush].
(This card can attack on the turn in which it is played.)

id:ST01-005
Name:Jinbe
Card type:Character
Cost:3
Attribute:Strike
Power:5000
Counter:-
Color:Red
Type:Fish-Man/Straw Hat Crew
Card Effect:
[DON!! x1] [When Attacking] Up to 1 of your Leader or Character cards other than this card gains +1000 power during this turn.


id:ST01-006
Name:Tony Tony.Chopper
Card type:Character
Cost:1
Attribute:Strike
Power:1000
Counter:-
Color:Red
Type:Animal/Straw Hat Crew
Card Effect:
[Blocker] (After your opponent declares an attack, you may rest this card to make it the new target of the attack.)


id:ST01-007
Name:Nami
Card type:Character
Cost:1
Attribute:Special
Power:1000
Counter:1000
Color:Red
Type:Straw Hat Crew
Card Effect:
[Activate: Main] [Once Per Turn] Give up to 1 rested DON!! card to your Leader or 1 of your Characters.


id:ST01-008
Name:Nico Robin
Card type:Character
Cost:3
Attribute:Wisdom
Power:5000
Counter:1000
Color:Red
Type:Straw Hat Crew
Card Effect:



id:ST01-009
Name:Nefeltari Vivi
Card type:Character
Cost:2
Attribute:Slash
Power:4000
Counter:1000
Color:Red
Type:Alabasta
Card Effect:




id:ST01-010
Name:Franky
Card type:Character
Cost:4
Attribute:Strike
Power:6000
Counter:1000
Color:Red
Type:Straw Hat Crew
Card Effect:



id:ST01-011
Name:Brook
Card type:Character
Cost:2
Attribute:Slash
Power:3000
Counter:2000
Color:Red
Type:Straw Hat Crew
Card Effect:
[On Play] Give up to 2 rested DON!! cards to your Leader or 1 of your Characters.


id:ST01-012
Name:Monkey.D.Luffy
Card type:Character
Cost:5
Attribute:Strike
Power:6000
Counter:-
Color:Red
Type:Supernovas/Straw Hat Crew
Card Effect:
[Rush] (This card can attack on the turn in which it is played.)
[DON!! x2] [When Attacking] Your opponent cannot activate [Blocker] during this battle.


id:ST01-013
Name:Roronoa Zoro
Card type:Character
Cost:3
Attribute:Slash
Power:5000
Counter:-
Color:Red
Type:Supernovas/Straw Hat Crew
Card Effect:
[DON!! x1] This Character gains +1000 power.


id:ST01-014
Name:Guard Point
Card type:Event
Cost:1
Attribute:-
Power:-
Counter:-
Color:Red
Type:Animal/Straw Hat Crew
Card Effect:
[Counter] Up to 1 of your Leader or Character cards gains +3000 power during this battle.
Trigger:
[Trigger] Your Leader or 1 of your Characters gains +1000 power during this turn.


id:ST01-015
Name:Gum-Gum Jet Pistol
Card type:Event
Cost:4
Attribute:-
Power:-
Counter:-
Color:Red
Type:Supernovas/Straw Hat Crew
Card Effect:
[Main] K.O. up to 1 of your opponent's Characters with 6000 power or less.
Trigger:
[Trigger] Activate this card's [Main] effect.


id:ST01-016
Name:Diable Jambe
Card type:Event
Cost:1
Attribute:-
Power:-
Counter:-
Color:Red
Type:Straw Hat Crew
Card Effect:
[Main] Select up to 1 of your {Straw Hat Crew} type Leader or Character cards. Your opponent cannot activate [Blocker] if that Leader or Character attacks during this turn.
Trigger:
[Trigger] K.O. 1 of your opponent's [Blocker] Characters with a cost of 3 or less.


id:ST01-017
Name:Thousand Sunny
Card type:Stage
Cost:2
Attribute:-
Power:-
Counter:-
Color:Red
Type:Straw Hat Crew
Card Effect:
[Activate: Main] You may rest this Stage: Up to 1 {Straw Hat Crew} type Leader or Character card on your field gains +1000 power during this turn.
"""

gameRuleContext = """
#Game Context for 'one piece' trading card game

##Game Objective:
*Reduce your opponent's life to zero or exhaust their deck so they can't draw a card at the start of their turn.

##Deck Building:
*A deck typically contains 50 cards. Decks consist of character cards, event cards, and stage cards.
*A separate "Leader" card represents the player's character.

##Game Setup:
*Each player starts with a Leader card and a deck.
*Players draw five cards as their starting hand.
*Players have a set amount of "Life" points based on their Leader card's specifications.

##Playing Areas
###Leader Area:
*Where your Leader Card goes at the start of the game.
*Remains here throughout the game.
###Character Area:
*Where you play your Character Cards.
*You can only play max 5 Character Cards on this Area.
*They refresh (stand up) at the start of your turn.
###DON Area:
*Where played DON Cards go.
*DON here are used as currency or cost to play other cards.
*They refresh (stand up) at the start of your turn.
###Other Important Areas:
*Deck: Your draw pile, placed face-down.
*Trash: Where cards go when used or KO'd.
*Life: Not a physical area, but each player's remaining Life is tracked, usually on a separate playmat or using a die.

##Turn Structure:
###Draw Phase:
*Draw one card from the deck.
###Refresh Phase:
*Reset any used cards.
###Main Phase:
*Rest DON and use it as resouse to play characters, events, and stages.
example 1:
You have a character "A" with 3 cost and you have 5 active DON in DON Area. Also your Character Area is empty.
You can rest 3 active DON as cost and play character "A" in Character Area.
After play character "A" in Character Area. You will have 3 rested DON and 2 active DON remain the Don Area.
example 2:
You have a character "A" with 3 cost and you have 2 active DON in DON Area. Also your Character Area is empty.
You checked your DON Area and there is no more than 3 active DON.
You dont have enough Don as resource/cost to play character "A".
Therefore you cannot play character "A".
*Players can choose to delare end turn.
When player finished all the move in his turn.
Player will declare end turn.
Current turn will end and pass to next player.
End turn without any move can also be a good strategy to save the card in hand for future turn.
*Players can declare attack to opponent's leader / rested character.
Declaring an Attack involve the below step:
1.Choose an Attacker:
Select a character or your leader card to attack with. Ensure the card is in an active (upright) position.
2.Select a Target: Declare that you are attacking the opponent's leader.
Note that you can't attack your opponent's active characters unless a card effect allows it.
3.Attach Don Cards: You can attach active Don cards from your DON area to your character or leader to boost their power.
This is optional but can make your attack stronger. When attaching 1 DON to your character or leader, its power will increase by 1000.
4.Compare Power: After declaring the attack and attaching Don cards, compare the total power of your attacker with your target.
5.Opponent's Response: The opponent can choose to block the attack (if they have a blocker character) or use counter cards from their hand to increase their leader's power temporarily.
6.Resolve the Attack: If your attacker's power is equal to or greater than the opponent's leader's power after any counters, the attack is successful. The opponent loses one life card.
Example 1
Your Turn: You have a character card with 5,000 power and have 2 active DON in DON Area. You attach 2 Don cards, each providing 1,000 power, for a total of 7,000 power.
Opponent's Leader: The opponent's leader has a power of 6,000.
Attack Declaration: You declare an attack on the opponent's leader using your character card.
Opponent's Response: The opponent decides not to use any counters or blockers.
Result: Your attack is successful because 7,000 (your total power) is greater than 6,000 (opponent's leader's power). The opponent loses one life card.

###Battle Phase:
*When player declare an attach, it will enter battle phase.
###End Phase:
*End your turn.

##Tips:
You have 1 active DON card, so you can rest it to play a character card with a cost of 1.
You have 2 active DON card, so you can rest it to play a character card with a cost of 2 or two character with cost 1.
Always consider the usage of DON and becareful that you should not play any character without paying enough cost of DON

"""

currentEnvContext = """
#Current Game Environment:
Current Turn : 1 //assume this is the first turn of the game
Current Turn player : Player 1
Player 1's Leader Area: ST01-001 (Active)
Player 1's Character Area: Empty
Player 1's Card in Hand : ['ST01-008', 'ST01-008', 'ST01-005', 'ST01-002', 'ST01-016']
Player 1's Life : 5
Player 1's Don Area: 1 active Don , 0 rested Don
Player 2's Leader Area: ST01-001 (Active)
Player 2's Character Area: Empty
Player 2's Card in Hand : 5 cards
Player 2's Life : 5
Player 2's Don Area: 0 active Don , 0 rested Don

"""

exploreGameTaskPrompt = cardListContext+ gameRuleContext + currentEnvContext + """

#Task :
You are now playing 'one piece' trading card game and you are a player 1.
You want to explicitly understand the card in different Area in the game.
Therefore, you will map the card details including all card attribute from card list for the card in different Area and card in hand.
Also supplement the card effect.
The card in Hand also need such details
With this, give a details summary and description in the Current Game Environment.
Only give description on the environment without providing other information.
"""


llm = Groq(model="llama3-8b-8192", api_key="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")

#agent = OpenAIAgent.from_tools([], llm=llm, verbose=True, context=context)
currentTurnEnvironment = llm.complete(exploreGameTaskPrompt)
turnPrompt = """
Current Turn Environment visible to Player 1

""" + currentTurnEnvironment.text



bestMovePrompt =cardListContext+ gameRuleContext + turnPrompt + """


#Task :
You are now playing 'one piece' trading card game and you are a player 1. Based on Current Turn Environment visible to Player 1 and the game rules,
1.analysis the card in your hand and different Area in the game.
Suggest and analysis all the legal and valid move following the game rules.

2.suggest which is the best move.

"""

result = llm.complete(bestMovePrompt)
print(result)

**Analysis of the Current Turn Environment:**

* Player 1's Leader Area: Monkey.D.Luffy (ST01-001) with 5 Life, 5000 Power, and the ability to give up to 1 rested DON!! card.
* Player 1's Character Area: Empty.
* Player 1's Card in Hand: 5 cards, including Nico Robin (ST01-008), Jinbe (ST01-005), Usopp (ST01-002), Diable Jambe (ST01-016), and another Jinbe (ST01-005).
* Player 1's Don Area: 1 active Don, 0 rested Don.

**Legal and Valid Moves:**

1. Resting a Don to play a character: Since there is only 1 active Don, Player 1 can rest it to play a character with a cost of 1.
2. Playing a character: Player 1 can play a character from their hand, but they need to ensure they have enough Don as resource/cost.
3. Using a card effect: Player 1 can use the card effects of the cards in their hand, such as Nico Robin's ability to give up to 1 rested DON!! card or Jinbe's ability to gain +1000 power when attacking.
4. Declaring an attack: Player 1 can declare an attack on the opponent's leader 